# Computing the potential modes up to high eccentricities through numerical fitting

First we must look at how the potential modes are prescribed

to do this we must first understand that the gravitational potential is given by the law

Phi = -GM1/|r1 - r|- GM2/|r2 - r|

we are purely interested in the fitting of this potential to hansen coefficients and therefore do not need to consider the 
case of inclined disk-binary as the wigner matrices will take care of the transferring of the inclined potential to that of the 
coplanar one. This takes care of the eccentric terms.

Additionally, we are not interested in the radial part of the potential since our study relates to the azimuthal modes. Computing the full potential 
may corrupt the azimuthally asymmetric parts of the potential as they drop at orders much higher than the monopole spherical mode.

Therefore, we are purely interested in transforming, at any multipole mode, the azimuthally varing parts of the potential to a series in harmonics of time.

From the spherical harmonic decomposition of the multipole potential one arrives at the conclusion that the part of interest is, for a multipole mode $l$ and azimuthal harmonic mode $m$

$$
\sum_{\mu}r_{12}^l\cos(m\phi - \mu\phi') = \sum_{\mu}\sum_{n} C^l_{\mu n} \cos(m\phi - (\mu + n)\Omega_b t)
$$

To do this, we must first, for a full binary rotation solve for the radius and the azimuth of the binary. The first step in doing so is solving the true anomaly.

In [1]:
## Solve the Kepler problem numerically
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as scopt
import scipy.integrate as scint


def Eccentric_Anomaly_Series(e):
    M = np.arange(0,1.01,0.01)*2*np.pi
    func = lambda E,MA,ecc: MA - E + ecc*np.sin(E)
    dfunc = lambda E,MA,ecc: -1 + ecc*np.cos(E)
    eccentric_anomalies = scopt.newton(func,M,dfunc,args = (M,e))
    return eccentric_anomalies

print(Eccentric_Anomaly_Series(0.1))

[0.         0.06980687 0.13957603 0.20927016 0.2788527  0.34828819
 0.41754266 0.48658387 0.55538158 0.6239078  0.69213691 0.76004585
 0.82761415 0.89482399 0.9616602  1.02811025 1.09416416 1.1598144
 1.22505583 1.28988554 1.35430273 1.41830854 1.48190595 1.54509961
 1.60789566 1.67030167 1.73232642 1.79397981 1.85527276 1.91621705
 1.97682524 2.03711059 2.09708693 2.15676863 2.21617048 2.27530767
 2.3341957  2.39285036 2.45128765 2.50952379 2.56757513 2.62545816
 2.6831895  2.74078582 2.79826388 2.8556405  2.91293252 2.97015682
 3.02733033 3.08446996 3.14159265 3.19871534 3.25585497 3.31302848
 3.37025279 3.42754481 3.48492143 3.54239949 3.59999581 3.65772714
 3.71561018 3.77366152 3.83189765 3.89033495 3.9489896  4.00787764
 4.06701483 4.12641668 4.18609837 4.24607472 4.30636007 4.36696826
 4.42791255 4.4892055  4.55085889 4.61288364 4.67528964 4.7380857
 4.80127935 4.86487677 4.92888258 4.99329976 5.05812948 5.12337091
 5.18902115 5.25507505 5.3215251  5.38836132 5.45557116 5.523139

In [2]:
def get_potential_function(l,mu,e):
    eccentric_anomalies = Eccentric_Anomaly_Series(e)
    binary_separation = 1 - e*np.cos(eccentric_anomalies)
    prefac = np.sqrt((1+e)/(1-e))
    azimuthal_anomaly = 2*np.arctan2(prefac*np.sin(eccentric_anomalies/2),np.cos(eccentric_anomalies/2))
    return (binary_separation**l)*np.cos(mu*azimuthal_anomaly),(binary_separation**l)*np.sin(mu*azimuthal_anomaly)

print(get_potential_function(2,2,0.1))

(array([ 0.81      ,  0.80080557,  0.77342796,  0.72847819,  0.66695501,
        0.59021637,  0.49994122,  0.39808317,  0.28681807,  0.16848727,
        0.04553882, -0.0795314 , -0.20423692, -0.32615449, -0.44297326,
       -0.55253826, -0.65288747, -0.74228205, -0.81922964, -0.88250085,
       -0.93113912, -0.96446452, -0.98207206, -0.98382503, -0.96984413,
       -0.94049309, -0.89636129, -0.83824424, -0.76712226, -0.68413807,
       -0.59057364, -0.48782679, -0.37738786, -0.2608167 , -0.13972031,
       -0.01573128,  0.10951283,  0.23438898,  0.35730703,  0.47672682,
        0.59117402,  0.69925464,  0.79966823,  0.89121978,  0.97283032,
        1.04354616,  1.10254694,  1.14915232,  1.18282751,  1.20318747,
        1.21      ,  1.20318747,  1.18282751,  1.14915232,  1.10254694,
        1.04354616,  0.97283032,  0.89121978,  0.79966823,  0.69925464,
        0.59117402,  0.47672682,  0.35730703,  0.23438898,  0.10951283,
       -0.01573128, -0.13972031, -0.2608167 , -0.37738786, -0.4

In [23]:
def get_coefficients(l,mu,n,e):
    func_cos, func_sin = get_potential_function(l,mu,e)
    MA = np.arange(0,1.01,0.01)*2*np.pi
    cos_series, sin_series = np.cos((mu+n)*MA), np.sin((mu+n)*MA)
    fs_cos = func_cos*cos_series
    fs_sin = func_sin*sin_series
    Clmun = (scint.trapezoid(fs_sin,x = MA) + scint.trapezoid(fs_cos,x =MA))/2/np.pi
    return Clmun

l , mu = 2,0
ML_Clmu1 = lambda l,mu,e: 0.5*e*(-l + 2*mu)
ML_Clmu2 = lambda l,mu,e: 0.125*e*e*(l*l - (3 + 4*mu)*l + 4*mu*mu + 5*mu)
ML_Clmu3 = lambda l,mu,e: (1/48)*e*e*e*(-l*l*l + 3*(3 + 2*mu)*l*l - (12*mu*mu + 33*mu + 17)*l + 2*mu*(4*mu*mu + 15*mu + 13))
    
import time
time_start = time.process_time()
print(get_coefficients(2,2,0,0.1))
time_end1 = time.process_time()
print(ML_Clmu3(2,-1,0.1))
time_end2 = time.process_time()

print("computational time for numerical:", time_end1 - time_start)
print("computational time for analytical:", time_end2 - time_end2)

0.9751435246753605
0.0001666666666666667
computational time for numerical: 0.0
computational time for analytical: 0.0
